In [1]:
#Author: Deidre Mensah
#Date: 06/09/2019
#Get places according to your search query using Google Places API

#import modules
import requests,json
import csv
import time
import requests_cache
from more_itertools import unique_everseen

<blockquote> <br> The following cell will create a new file called "Place_Search_orig.csv" where your results will output to. <br><b>You may need to run the following cell twice because there may be a lock on the file when initially run.</b></blockquote>

In [3]:
#opens your csv
output = open('Place_Search_orig.csv', "w", newline='')

#defines fields of csv
writer = csv.DictWriter(output, fieldnames=["Name", "Address", "Lat", "Long"])

#writes the field names to a csv
writer.writeheader()

In [4]:
# enter your api key here
api_key = 'YOUR_API_GOES_HERE'

In [5]:
#create database to store cached results to
requests_cache.install_cache("results_cache")

<blockquote> After running the following cell, you will be prompted to enter your query.<br> A file named "Place_Search_final.csv" will be be created in the directory where you saved this script.<b><br>You can run this cell for multiple times for different searches before you delete the duplicates and download the csv in the final cell.</blockquote></b>

In [6]:
# url we make a request from
url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"

# query to search google by
query = input('Search query: ')

# return response object from first page of results
response = requests.get(url + 'query=' + query + '&key=' + api_key)

# pause for 2 seconds to give time for page to be accessible
time.sleep(2)

# json format data into python format data
results = response.json()

#gets results dictionary that was nested in the list of other dictionaries based on key value
results_all = results["results"]

print("Places downloading...")

#if we only have 1 page of results
if "next_page_token" not in results:
    for result in results_all:
        name = (result["name"])
        address = (result["formatted_address"])
        lat = str((result["geometry"]["location"]["lat"]))
        long = str((result["geometry"]["location"]["lng"]))
        row = {"Name": str(name), "Address": str(address), "Lat": str(lat), "Long": str(long)}
        writer.writerow(row)
        print (str(name) + ", " + str(address) + ", " + str(lat) + ", " + str(long))
        
#if there is a 2nd page we can acccess
elif "next_page_token" in results:
    nextpagetoken = results["next_page_token"]
    # get the response object of the 2nd page of results
    response_2 = requests.get(url + 'pagetoken=' + nextpagetoken + '&key=' + api_key)
    # pause for 2 seconds to give time for page to be accessible
    time.sleep(2)
    # convert 2nd page to json
    results_2 = response_2.json()
    # get result dictionary
    results_all_2 = results_2['results']
    
    # all of our dictionaries from our results will be added to this list
    total_results = []
    # extends the first page of our response which a list of dictionaries by dictionaries from page 2
    results_all.extend(results_all_2)

    # grabs each dictionary in the list
    for dict in results_all:
        # if a dictionary doesnt appear in the list
        if dict not in total_results:
            # appends dictionary to list
            total_results.append(dict)

    for result in results_all:
        name = (result["name"])
        address = (result["formatted_address"])
        lat = str((result["geometry"]["location"]["lat"]))
        long = str((result["geometry"]["location"]["lng"]))
        row = {"Name": name, "Address": address, "Lat": lat, "Long": long}
        writer.writerow(row)
        
    #if there is no 3rd page we can access
    if "next_page_token" not in results_2:
        response_2 = requests.get(url + 'pagetoken=' + nextpagetoken + '&key=' + api_key)
        time.sleep(2)
        results_2 = response_2.json()
        results_all_2 = results_2['results']
        total_results = []
        results_all.extend(results_all_2)

        for dict in results_all:
            if dict not in total_results:
                total_results.append(dict)

        for result in results_all:
            name = (result['name'])
            address = (result["formatted_address"])
            lat = str((result["geometry"]["location"]["lat"]))
            long = str((result["geometry"]["location"]["lng"]))
            row = {"Name": name, "Address": address, "Lat": lat, "Long": long}
            writer.writerow(row)
            print (str(name) + ", " + str(address) + ", " + str(lat) + ", " + str(long))
            
    #if there is a 3rd page we can access
    elif "next_page_token" in results_2:
        # get next page token from dictionary
        nextpagetoken_2 = results_2["next_page_token"]
        # get the response object of the 3rd page of results - NO NEXT PAGE TOKEN BECAUSE YOU CAN ONLY REQUEST UP TO 60
        response_3 = requests.get(url + 'pagetoken=' + nextpagetoken_2 + '&key=' + api_key)
        time.sleep(2)
        results_3 = response_3.json()
        results_all_3 = results_3['results']

        # extending the list of dictionaries again with the third page of dictionaries
        total_results.extend(results_all_3)

        # grabs each dictionary in the list
        for dict in total_results:
            #if a dictionary doesnt exist in the list
            if dict not in total_results:
                # appends dictionaries by what we extended our list by
                total_results.append(dict)

        for result in total_results:
            name = (result['name'])
            address = (result["formatted_address"])
            lat = str((result["geometry"]["location"]["lat"]))
            long = str((result["geometry"]["location"]["lng"]))
            row = {"Name": str(name), "Address": str(address), "Lat": str(lat), "Long": str(long)}
            writer.writerow(row)
            print (str(name) + ", " + str(address) + ", " + str(lat) + ", " + str(long))

print("PLACES HAVE BEEN DOWNLOADED.")

Search query: barry's bootcamp
Places downloading...
Barry's UES, 1216 2nd Ave, New York, NY 10065, United States, 40.7634988, -73.9620226
Barry's Park Avenue South, 107 E 27th St, New York, NY 10016, United States, 40.7425935, -73.9839989
Barry's Chelsea, 135 W 20th St, New York, NY 10011, United States, 40.74185200000001, -73.995577
Barry's Noho, 419 Lafayette St, New York, NY 10003, United States, 40.72861109999999, -73.9922222
Barry's Scarsdale, 1142 Wilmot Rd, Scarsdale, NY 10583, United States, 40.9851586, -73.7767483
Barry's TriBeCa, 1 York St, New York, NY 10013, United States, 40.7211592, -74.00552669999999
Barry's UWS, 72-74 W 69th St, New York, NY 10023, United States, 40.77522159999999, -73.9799151
Barry's Bootcamp Norwalk, Norwalk, CT 06854, United States, 41.1046669, -73.4190062
PLACES HAVE BEEN DOWNLOADED.


<blockquote> <br> <b>You may need to run the following cell twice to delete duplicates and output into final csv.</b></blockquote>

In [8]:
#deletes duplicates
with open('Place_Search_orig.csv','r') as file, open('Place_Search_final.csv','w') as out_file:
    out_file.writelines(unique_everseen(file))
    
output.close()

print ("csv download complete")

csv download complete
